In [1]:
#这里的代码是使用lightgbm 对预测dt的特征做特征重要性排序。

import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.cross_validation import train_test_split


pd.options.mode.chained_assignment = None  # default='warn'

train_list = ['180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
          '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
          '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
          '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
          '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021',\
           '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029','181101',\
             '181030','181031','181101' ]

valid_list = ['181102','181103','181104','181105', '181106', '181107', '181108', '181109','181110']


day_all = ['180501', '180502', '180503', '180504', '180505', '180506', '180507', '180508', '180509', '180510',\
           '180511', '180512', '180513', '180514', '180515', '180516', '180517', '180518', '180519', '180520',\
           '180521', '180522', '180523', '180524', '180525', '180526', '180527', '180528', '180529', '180530',\
           '180531','180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
           '180711', '180712', '180713', '180714', '180715',\
           '180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725',\
           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804',\
           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822', \
            '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
            '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021',\
           '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030', \
           '181031', '181101', '181102', '181103', '181104', '181105', '181106', '181107', '181108', '181109',\
           '181110', '181111']

c:\program files\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def over_data_do(s):
    if s<0:
        return 0
    elif s> 2880:
        return 2880
    else:
        return s

In [3]:
#不用重复读取按天数据 如果执行这条命令，那么上下命令都不用执行
n_i = 0
train = pd.read_csv('./data_dt_train/dt_split_%d.csv'%n_i)

train = train.round(3)

cols = ['day_bef_%d_dt'%i for i in range(1,8)]

train[cols].fillna(method="ffill",axis=1,inplace=True)
train[cols].fillna(method="bfill",axis=1,inplace=True)


train['dt_std'] = train[cols].apply('std',axis=1)
train['dt_mean'] = train[cols].apply('mean',axis=1)
train['dt_median'] = train[cols].apply('median',axis=1)

In [25]:
import json
import lightgbm as lgb

str_cols = ['order_id','job_name','day_bef_0_dt']
features_all_merge = pd.read_csv(r'dt_feature_score1120.csv')
features_all_merge = features_all_merge['feature'][:150]
#read_cols = list(set(str_cols+features_all))
read_cols = list(set(str_cols+list(features_all_merge)))


In [29]:
#读取数据 按天

# 划分训练集， 验证集 测试集

train_df = []
n_i = 0  #选取第几轮数据切片
for day in train_list:
    print(day,end=' ')
    day_all_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train_dt\with_res_%s_dt.csv'%day,nrows=5000)
    
    #day_all_data = day_all_data[day_all_data.end_time.notnull()]
    
    #过滤结束日期超过2天的
    day_after_2 = day_all[day_all.index(day)+1]
    day_after_2_int = int('20'+day_after_2+'235959')
    day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
    
    #train_df.append(day_all_data.sample(frac=0.1))

    train_df.append(day_all_data)
    
    del day_all_data
train = pd.concat(train_df, ignore_index=True)

#train.to_csv('./data_dt_train/dt_split_%d.csv'%n_i,index=False)

valid_df = []
for day in valid_list:
    print(day,end=' ')
    day_all_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train_dt\with_res_%s_dt.csv'%day,nrows=6000)
    #day_all_data = day_all_data[day_all_data.end_time.notnull()]
    day_after_2 = day_all[day_all.index(day)+1]
    day_after_2_int = int('20'+day_after_2+'235959')
    day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
    
    valid_df.append(day_all_data)
    del day_all_data
valid = pd.concat(valid_df, ignore_index=True)

# test_df = []

# for day in test_list:
#     print(day,end=' ')   
#     day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train/train_%s_edt.csv'%day)
#     day_all_data = day_all_data[day_all_data.end_time.notnull()]
#     test_df.append(day_all_data)
    
# test = pd.concat(test_df, ignore_index=True)

#空值处理

print(train.shape)
print(valid.shape)

180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 181011 181012 181013 181014 181015 181016 181017 181018 181019 181020 181021 181022 181023 181024 181025 181026 181027 181028 181029 181101 181030 181031 181101 181102 181103 181104 181105 181106 181107 181108 181109 181110 (329080, 280)
(33515, 280)


In [30]:

train_y = train_y.fillna(train_y.mean())
valid_y = valid_y.fillna(valid_y.mean())
test_y = test_y.fillna(test_y.mean())

In [31]:
#cols =['day_bef_1_dt','day_bef_3_dt','day_bef_2_dt']
cols = ['day_bef_%d_dt'%i for i in range(1,8)]

train[cols].fillna(method="ffill",axis=1,inplace=True)
train[cols].fillna(method="bfill",axis=1,inplace=True)

valid[cols].fillna(method="ffill",axis=1,inplace=True)
valid[cols].fillna(method="bfill",axis=1,inplace=True)


train['dt_std'] = train[cols].apply('std',axis=1)
valid['dt_std'] = valid[cols].apply('std',axis=1)

train['dt_mean'] = train[cols].apply('mean',axis=1)
valid['dt_mean'] = valid[cols].apply('mean',axis=1)

train['dt_median'] = train[cols].apply('median',axis=1)
valid['dt_median'] = valid[cols].apply('median',axis=1)

In [32]:
cols_train =[clm for clm in valid.columns if clm not in ['order_id','job_name','v_date','order_d','from_time_1000_start',\
                                                            'start_time','end_time','order_id','job_name','day_0_bef_start_time',\
                                                            'day_0_bef_end_time','day_bef_0_dt']]


valid_x = valid[cols_train]
valid_y = valid['day_bef_0_dt']

train_x,test_x,train_y,test_y = train_test_split(train[cols_train],train['day_bef_0_dt'],random_state=33,test_size=0.2)

# train_y = train_y.fillna(train_y.mean())
# valid_y = valid_y.fillna(valid_y.mean())
# test_y = test_y.fillna(test_y.mean())

print(train_x.shape)
print(test_x.shape)
print(valid_x.shape)

(263264, 273)
(65816, 273)
(33515, 273)


In [34]:
train_y = train_y.fillna(0)
valid_y = valid_y.fillna(0)
test_y = test_y.fillna(0)

In [35]:
# lightgbm 算法预测
import json
import lightgbm as lgb

y_train = train_y.values
y_test = test_y.values

X_train = train_x[features_all_merge].values
X_test = test_x[features_all_merge].values

# X_train = train_x.values
# X_test = test_x.values

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train,silent=True)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


# specify your configurations as a dict
params = {'task': 'train',
   'boosting_type': 'gbdt',
   'objective': 'regression',
   'metric':  'mae',
   'num_leaves': 2**5,
   'learning_rate': 0.1,
   'feature_fraction': 0.9,
   'bagging_fraction': 0.9,
   'bagging_freq': 5,
   'verbose': 50,
    'verbose_eval':50}
  
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_eval,
                early_stopping_rounds=10,
                verbose_eval = False )

print('Save model...')

# from sklearn.externals import joblib
# joblib.dump(gbm,'gbm.pkl')
# clf = joblib.load("gbm.pkl")

# save model to file

# gbm.save_model('lightgbm_model.txt')
print('Start predicting...')

# predict

y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
vlid_pred = gbm.predict(valid_x[features_all_merge], num_iteration=gbm.best_iteration)

# eval print(y_pred)
print('The mean_absolute_error of prediction train_data is:', mean_absolute_error(y_test, y_pred))
print('The mean_absolute_error of prediction valid_data is:', mean_absolute_error(valid_y, vlid_pred))

print('The mean_absolute_error of prediction train_data is:', mean_squared_error(y_test, y_pred))
print('The mean_absolute_error of prediction valid_data is:', mean_squared_error(valid_y, vlid_pred))

Start training...
Save model...
Start predicting...
The mean_absolute_error of prediction train_data is: 41.4129494545
The mean_absolute_error of prediction valid_data is: 55.5874874698
The mean_absolute_error of prediction train_data is: 12069.2319172
The mean_absolute_error of prediction valid_data is: 17981.4488253


In [ ]:
The mean_absolute_error of prediction train_data is: 43.7076170557
The mean_absolute_error of prediction valid_data is: 59.1640163048
The mean_absolute_error of prediction train_data is: 12906.7405493
The mean_absolute_error of prediction valid_data is: 19853.1192745

In [37]:
# The mean_absolute_error of prediction train_data is: 27.6253644104
# The mean_absolute_error of prediction valid_data is: 42.1227454821

### 特征选择   保存模型特征重要性
df = pd.DataFrame(train_x[features_all_merge].columns.tolist(), columns=['feature'])
df['importance']=list(gbm.feature_importance())                           # 特征分数
df = df.sort_values(by='importance',ascending=False)                      # 特征排序
df.to_csv("dt_feature_score1121.csv",index=None,encoding='gbk')  # 保存分数

In [17]:
# lightgbm 算法预测
import json
import math
import lightgbm as lgb
# y_train = train_y.values
# y_test = test_y.values
features = pd.read_csv("dt_feature_score.csv") 
feature_imports = features['feature'][:100]

### 选择筛选的重要特征训练预测

In [27]:
# lightgbm 算法预测


# y_train = train_y.apply(lambda x:math.log(1+x)).values
# y_test = test_y.apply(lambda x:math.log(1+x)).values
#y_test = test_y.values

y_train = train_y.values
y_test = test_y.values

X_train = train_x.values
X_test = test_x.values


X_train = train_x[feature_imports].values
X_test = test_x[feature_imports].values

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train,silent=True)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


# specify your configurations as a dict
params = {'task': 'train',
   'boosting_type': 'gbdt',
   'objective': 'regression',
   'metric':  'mse',
   'num_leaves': 2**6,
   'learning_rate': 0.1,
   'feature_fraction': 0.9,
   'bagging_fraction': 0.8,
   'bagging_freq': 5,
   'verbose': 50,
    'verbose_eval':50}
  
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=110,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

print('Save model...')

# from sklearn.externals import joblib
# joblib.dump(gbm,'gbm.pkl')
# clf = joblib.load("gbm.pkl")

# save model to file

gbm.save_model('lightgbm_model.txt')
print('Start predicting...')

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
vlid_pred = gbm.predict(valid_x[feature_imports], num_iteration=gbm.best_iteration)


# y_pred = pd.Series(y_pred).apply(lambda x:math.exp(x)-1)
# vlid_pred =  pd.Series(vlid_pred).apply(lambda x:math.exp(x)-1)

# y_test = pd.Series(y_test).apply(lambda x:math.exp(x)-1)


# eval print(y_pred)
print('The mean_absolute_error of prediction train_data is:', mean_absolute_error(y_test, y_pred))
print('The mean_absolute_error of prediction valid_data is:', mean_absolute_error(valid_y, vlid_pred))

Start training...
[1]	valid_0's l2: 93537.3
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 79133.8
[3]	valid_0's l2: 67286.9
[4]	valid_0's l2: 57719.9
[5]	valid_0's l2: 49860.7
[6]	valid_0's l2: 43602.6
[7]	valid_0's l2: 38322
[8]	valid_0's l2: 33961.8
[9]	valid_0's l2: 30460
[10]	valid_0's l2: 27535.6
[11]	valid_0's l2: 25216.3
[12]	valid_0's l2: 23177.9
[13]	valid_0's l2: 21474.2
[14]	valid_0's l2: 20092.6
[15]	valid_0's l2: 18882.7
[16]	valid_0's l2: 17890.2
[17]	valid_0's l2: 17068.4
[18]	valid_0's l2: 16371.5
[19]	valid_0's l2: 15764.4
[20]	valid_0's l2: 15221.2
[21]	valid_0's l2: 14805.6
[22]	valid_0's l2: 14430.3
[23]	valid_0's l2: 14127.4
[24]	valid_0's l2: 13869
[25]	valid_0's l2: 13637.3
[26]	valid_0's l2: 13427.5
[27]	valid_0's l2: 13247.9
[28]	valid_0's l2: 13035.9
[29]	valid_0's l2: 12882.6
[30]	valid_0's l2: 12748
[31]	valid_0's l2: 12635.9
[32]	valid_0's l2: 12520.3
[33]	valid_0's l2: 12415.8
[34]	valid_0's l2: 12315.4
[35]	valid_0's l2: 

In [28]:
print('The mean_absolute_error of prediction train_data is:', mean_squared_error(y_test, y_pred))
print('The mean_absolute_error of prediction valid_data is:', mean_squared_error(valid_y, vlid_pred))

The mean_absolute_error of prediction train_data is: 9650.73137949
The mean_absolute_error of prediction valid_data is: 8237.63599525


features个数:
    150
    The mean_absolute_error of prediction train_data is: 37.1458226484
    The mean_absolute_error of prediction valid_data is: 35.8024545072
    115
    The mean_absolute_error of prediction train_data is: 37.3499854566
    The mean_absolute_error of prediction valid_data is: 35.3970497482
    100
    The mean_absolute_error of prediction train_data is: 37.3063873753
    The mean_absolute_error of prediction valid_data is: 35.5408513606
    90
    The mean_absolute_error of prediction train_data is: 37.3364645378
    The mean_absolute_error of prediction valid_data is: 35.5955937712


100个特征：
    src:
    The mean_absolute_error of prediction train_data is: 37.3063873753
    The mean_absolute_error of prediction valid_data is: 35.5408513606
    log:
    The mean_absolute_error of prediction train_data is: 42.2862326881
    The mean_absolute_error of prediction valid_data is: 35.1697839294
    
    The mean_absolute_error of prediction train_data is: 17160.1229476
    The mean_absolute_error of prediction valid_data is: 10126.7373996
    
    The mean_absolute_error of prediction train_data is: 9504.77582456
    The mean_absolute_error of prediction valid_data is: 8296.48791077

In [ ]:
import math
print(math.log(4))

print(math.exp(1.3862943611198906))

### 生产每日dt并保存

In [4]:
cols_train =[clm for clm in train.columns if clm not in ['order_id','job_name','v_date','order_d','from_time_1000_start',\
                                                            'start_time','end_time','order_id','job_name','day_0_bef_start_time',\
                                                            'day_0_bef_end_time','day_bef_0_dt']]


train_x,test_x,train_y,test_y = train_test_split(train[cols_train],train['day_bef_0_dt'],random_state=33,test_size=0.2)

train_y = train_y.fillna(train_y.mean())
test_y = test_y.fillna(test_y.mean())
print(train_x.shape)
print(test_x.shape)

(408816, 220)
(102204, 220)


In [5]:
test_y[test_y.isnull()]

Series([], Name: day_bef_0_dt, dtype: float64)

In [6]:
# lightgbm 算法预测
import json
import math
import lightgbm as lgb
# y_train = train_y.values
# y_test = test_y.values
features = pd.read_csv("dt_feature_score.csv") 
feature_imports = features['feature'][:100]

y_train = train_y.values
y_test = test_y.values

# X_train = train_x.values
# X_test = test_x.values


X_train = train_x[feature_imports].values
X_test = test_x[feature_imports].values

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train,silent=True)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


# specify your configurations as a dict
params = {'task': 'train',
   'boosting_type': 'gbdt',
   'objective': 'regression',
   'metric':  'mse',
   'num_leaves': 2**6,
   'learning_rate': 0.1,
   'feature_fraction': 0.9,
   'bagging_fraction': 0.8,
   'bagging_freq': 5,
   'verbose': 50,
    'verbose_eval':50}
  
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=110,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

print('Save model...')

from sklearn.externals import joblib
joblib.dump(gbm,'gbm.pkl')
#clf = joblib.load("gbm.pkl")

# save model to file

gbm.save_model('lightgbm_model.txt')
print('Start predicting...')

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred = pd.Series(y_pred).apply(lambda x:over_data_do(x))
# vlid_pred = gbm.predict(valid_x[feature_imports], num_iteration=gbm.best_iteration)


# y_pred = pd.Series(y_pred).apply(lambda x:math.exp(x)-1)
# vlid_pred =  pd.Series(vlid_pred).apply(lambda x:math.exp(x)-1)

# y_test = pd.Series(y_test).apply(lambda x:math.exp(x)-1)


# eval print(y_pred)
print('The mean_absolute_error of prediction train_data is:', mean_absolute_error(y_test, y_pred))
print('The mean_absolute_error of prediction train_data is:', mean_squared_error(y_test, y_pred))

Start training...
[1]	valid_0's l2: 93537.3
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 79133.8
[3]	valid_0's l2: 67286.9
[4]	valid_0's l2: 57719.9
[5]	valid_0's l2: 49860.7
[6]	valid_0's l2: 43602.6
[7]	valid_0's l2: 38322
[8]	valid_0's l2: 33961.8
[9]	valid_0's l2: 30460
[10]	valid_0's l2: 27535.6
[11]	valid_0's l2: 25216.3
[12]	valid_0's l2: 23177.9
[13]	valid_0's l2: 21474.2
[14]	valid_0's l2: 20092.6
[15]	valid_0's l2: 18882.7
[16]	valid_0's l2: 17890.2
[17]	valid_0's l2: 17068.4
[18]	valid_0's l2: 16371.5
[19]	valid_0's l2: 15764.4
[20]	valid_0's l2: 15221.2
[21]	valid_0's l2: 14805.6
[22]	valid_0's l2: 14430.3
[23]	valid_0's l2: 14127.4
[24]	valid_0's l2: 13869
[25]	valid_0's l2: 13637.3
[26]	valid_0's l2: 13427.5
[27]	valid_0's l2: 13247.9
[28]	valid_0's l2: 13035.9
[29]	valid_0's l2: 12882.6
[30]	valid_0's l2: 12748
[31]	valid_0's l2: 12635.9
[32]	valid_0's l2: 12520.3
[33]	valid_0's l2: 12415.8
[34]	valid_0's l2: 12315.4
[35]	valid_0's l2: 

In [27]:
from sklearn.externals import joblib
#joblib.dump(gbm,'gbm.pkl')
clf = joblib.load("gbm.pkl")
    

for day in valid_list:
    print(day,end=' ')   
    day_all_data = pd.read_csv(r'E:\jupyter\CTM\data_train\train_dt\with_res_%s_dt.csv'%day)
    #day_all_data = day_all_data[day_all_data.end_time.notnull()]
    day_after_2 = day_all[day_all.index(day)+1]
    day_after_2_int = int('20'+day_after_2+'235959')
    day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
    day_all_data.index = range(day_all_data.shape[0])
    
    day_all_data[cols].fillna(method="ffill",axis=1,inplace=True)
    day_all_data[cols].fillna(method="bfill",axis=1,inplace=True)

    day_all_data['dt_std'] = day_all_data[cols].apply('std',axis=1)
    day_all_data['dt_mean'] = day_all_data[cols].apply('mean',axis=1)
    day_all_data['dt_median'] = day_all_data[cols].apply('median',axis=1)
    
    valid_x = day_all_data[cols_train]
    valid_y = day_all_data['day_bef_0_dt']
    
    
    vlid_pred = gbm.predict(valid_x[feature_imports], num_iteration=gbm.best_iteration)
    
    vlid_pred = pd.Series(vlid_pred).apply(lambda x:over_data_do(x))
    
    
    pre_save = day_all_data[['order_id','job_name','day_bef_0_dt','end_time','day_bef_1_dt','day_bef_2_dt','day_bef_3_dt',\
                             'day_bef_4_dt','day_bef_5_dt','day_bef_6_dt','day_bef_7_dt','dt_std','dt_mean']]
    pre_save['pre_dt'] =  vlid_pred
    
    valid_y = valid_y.fillna(0)
    
    pre_save['error'] = pre_save[['pre_dt','day_bef_0_dt']].apply(lambda x:abs(x[0]-x[1]),axis=1)
    pre_save['hour'] = pre_save['end_time'].apply(str).apply(lambda x:x[6:10])
    print(pre_save.groupby('hour')['error'].mean())
    
    print(mean_absolute_error(valid_y, vlid_pred), mean_squared_error(valid_y, vlid_pred))
    
    pre_save = pre_save.round(2)
    pre_save.to_csv(r'E:\jupyter\CTM\data_train\pred_dt\pre_dt_%s.csv'%day,index=False)
    

181016 hour
1604      14.626209
1605      10.996173
1606      14.349526
1607      22.466835
1608      30.297117
1609      32.531320
1610      44.775179
1611      45.392138
1612      23.983516
1613      38.374382
1614      34.203325
1615      37.824540
1616      22.902829
1617      86.160106
1618     105.452067
1619      29.430587
1620      21.804924
1621      18.465749
1622      20.990285
1623       6.630062
1700      85.907528
1701       8.662566
1702      93.038218
1703     134.673934
1704      37.346492
1705      82.151691
1706     141.125539
1707     244.876839
1708     285.645014
1709     713.881240
1710     151.262785
1711     269.443183
1712     226.990240
1713     830.908492
1714     434.847033
1715     240.825329
1716     516.559146
1717    1641.174909
1719       2.088013
1720     364.007378
1722     122.381319
1723     506.576811
Name: error, dtype: float64
27.4722964166 6898.77847285
181017 hour
1704      31.214836
1705      37.176633
1706      29.169866
1707      39.659800


In [23]:
pre_save['error'] = pre_save[['pre_dt','day_bef_0_dt']].apply(lambda x:abs(x[0]-x[1]),axis=1)


In [25]:
pre_save['hour'] = pre_save['end_time'].apply(str).apply(lambda x:x[6:10])
pre_save.head()

,order_id,job_name,day_bef_0_dt,end_time,day_bef_1_dt,day_bef_2_dt,day_bef_3_dt,day_bef_4_dt,day_bef_5_dt,day_bef_6_dt,day_bef_7_dt,pre_dt,error,hour
0,0amh31,TRM_B971_CR_SECU_RELA,0.08,2.018102e+13,0.10,0.03,0.03,0.22,0.03,0.02,0.02,0.15,0.07,2116
1,5fuce2,SYS_FS_LD_TDW_4_P_HRRH_2_NSZCPSDB_SZ_BUD_BUD_A...,436.18,2.018102e+13,437.07,432.07,437.12,433.17,432.05,433.08,437.12,451.46,15.28,2107
2,0bvw51,EXZ_T_HTDW_PDM_T41_TDP2P_LOAN_CNCN_LST_RSK_DAT,0.18,2.018102e+13,1.70,9.00,0.23,1.37,4.45,0.70,3.40,5.35,5.17,2107
3,0bzsw1,EXZ_A_HC03_VBSDTA_KPPCSDTAP,421.60,2.018102e+13,410.85,517.82,490.35,502.67,602.85,600.73,735.72,436.61,15.01,2107
4,0afqg1,EXZ_P_HSCR_5_EMUSER_JOB_SD_MODIFY_BAC,0.03,2.018102e+13,0.03,0.03,0.03,0.27,0.80,0.02,0.03,0.71,0.68,2120


In [29]:
pre_save.groupby('hour')['error'].median()

hour
2104     2.410
2105     3.010
2106     3.980
2107     8.570
2108     5.320
2109     5.390
2110     5.680
2111     3.110
2112     3.230
2113     3.150
2114     1.610
2115     0.750
2116     0.820
2117     3.020
2118     2.860
2119     1.290
2120    49.270
2121    16.275
2122    28.925
2123     0.930
Name: error, dtype: float64